<a href="https://colab.research.google.com/github/misalsachin/deep_learning-RNN/blob/master/Fake_news_classifier_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#


In [36]:
import pandas as pd
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense,Dropout


In [2]:
df=pd.read_csv('/content/drive/My Drive/LSTM Kaggle fake news dataset/train.csv',engine='python', error_bad_lines=False)

Skipping line 12520: unexpected end of data


In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# Check Nan Value in title colum
df['title'].isnull().sum()

339

In [5]:
# Check Nan Value
df.isnull().values.any()

True

In [6]:
# Check Nan Value in whole train dataset
df.isnull().sum().sum()

1523

In [7]:
###Drop Nan Values
df=df.dropna()

In [8]:
# Deleted Nan/Null values from dataset
df.isnull().sum().sum()

0

In [9]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [10]:
## Get the Dependent features
y=df['label']

In [11]:
X.shape

(11022, 4)

In [12]:
y.shape

(11022,)

In [13]:
### fix the Vocabulary size
voc_size=5000

In [14]:
messages=X.copy()

In [15]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [16]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
messages.reset_index(inplace=True)

In [18]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [19]:
# Data preprocessing
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
#dataset preprocesssing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # remove all other than alphabet & add spece
    review = review.lower()    # convert the dataset in lower case
    review = review.split()    # split the word by space  
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [26]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [27]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4161, 2645, 4593, 719, 1523, 3311, 4275, 775, 4944, 123],
 [1484, 4986, 3709, 165, 4992, 3080, 2618],
 [297, 84, 1416, 1292],
 [3820, 100, 2819, 99, 3764, 4161],
 [4947, 4992, 1539, 4200, 1191, 406, 4992, 2517, 3104, 319],
 [4812,
  2172,
  3707,
  274,
  939,
  2086,
  3566,
  1457,
  3897,
  4895,
  3482,
  920,
  3983,
  4399,
  2618],
 [1490, 1203, 1697, 4891, 2839, 2099, 2961, 77, 4740, 4719, 466],
 [805, 2169, 4124, 1909, 425, 3646, 2086, 3300, 4740, 4719, 466],
 [1641, 3819, 3039, 3446, 2979, 2649, 289, 3025, 2086, 3006],
 [4310, 217, 3293, 3412, 2056, 4397, 4698, 1406],
 [4268, 4396, 1388, 1221, 87, 1203, 2871, 2824, 3144, 2179, 3484],
 [99, 3811, 1523, 2649, 2086, 425],
 [936, 731, 1295, 4410, 2605, 4511, 3568, 1687, 3436],
 [1884, 3418, 1909, 2027, 297, 792, 1475, 4740, 4719, 466],
 [255, 1818, 3341, 4837, 1722, 4740, 4719, 466],
 [452, 3297, 2923, 4923, 4887, 1834, 2958, 1701, 1969, 698],
 [2133, 3276, 4986],
 [3333, 549, 4232, 126, 2086, 1821, 701, 2618],
 [965, 2709, 370

In [28]:
# Embedding Representation
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  775 4944  123]
 [   0    0    0 ... 4992 3080 2618]
 [   0    0    0 ...   84 1416 1292]
 ...
 [   0    0    0 ... 2192 1135 1152]
 [   0    0    0 ... 2850 1318 2618]
 [   0    0    0 ... 4390 2645 4270]]


In [29]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4161,
       2645, 4593,  719, 1523, 3311, 4275,  775, 4944,  123], dtype=int32)

In [30]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
len(embedded_docs),y.shape


(11022, (11022,))

In [32]:
# converting to numpy array
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [34]:
# model Traning
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=60)

Epoch 1/10
124/124 [==============================] - 4s 35ms/step - loss: 0.3972 - accuracy: 0.8151 - val_loss: 0.2198 - val_accuracy: 0.9065
Epoch 2/10
124/124 [==============================] - 4s 32ms/step - loss: 0.1462 - accuracy: 0.9430 - val_loss: 0.1995 - val_accuracy: 0.9131
Epoch 3/10
124/124 [==============================] - 4s 32ms/step - loss: 0.0907 - accuracy: 0.9679 - val_loss: 0.2706 - val_accuracy: 0.8928
Epoch 4/10
124/124 [==============================] - 4s 31ms/step - loss: 0.0611 - accuracy: 0.9798 - val_loss: 0.2396 - val_accuracy: 0.9167
Epoch 5/10
124/124 [==============================] - 4s 32ms/step - loss: 0.0406 - accuracy: 0.9881 - val_loss: 0.3358 - val_accuracy: 0.9090
Epoch 6/10
124/124 [==============================] - 4s 31ms/step - loss: 0.0299 - accuracy: 0.9919 - val_loss: 0.4150 - val_accuracy: 0.9082
Epoch 7/10
124/124 [==============================] - 4s 31ms/step - loss: 0.0144 - accuracy: 0.9966 - val_loss: 0.2997 - val_accuracy: 0.9046

In [37]:
#Adding Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [38]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=60)

Epoch 1/10
124/124 [==============================] - 4s 35ms/step - loss: 0.4257 - accuracy: 0.7863 - val_loss: 0.2246 - val_accuracy: 0.8986
Epoch 2/10
124/124 [==============================] - 4s 32ms/step - loss: 0.1668 - accuracy: 0.9342 - val_loss: 0.1937 - val_accuracy: 0.9186
Epoch 3/10
124/124 [==============================] - 4s 32ms/step - loss: 0.1057 - accuracy: 0.9606 - val_loss: 0.1984 - val_accuracy: 0.9148
Epoch 4/10
124/124 [==============================] - 4s 32ms/step - loss: 0.0701 - accuracy: 0.9744 - val_loss: 0.2230 - val_accuracy: 0.9118
Epoch 5/10
124/124 [==============================] - 4s 31ms/step - loss: 0.0480 - accuracy: 0.9837 - val_loss: 0.2782 - val_accuracy: 0.9170
Epoch 6/10
124/124 [==============================] - 4s 31ms/step - loss: 0.0447 - accuracy: 0.9869 - val_loss: 0.3161 - val_accuracy: 0.9076
Epoch 7/10
124/124 [==============================] - 4s 31ms/step - loss: 0.0246 - accuracy: 0.9928 - val_loss: 0.3549 - val_accuracy: 0.9079

In [39]:

# Performance Metrics And Accuracy¶
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [40]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[1866,  212],
       [ 112, 1448]])

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9109400769653656

In [42]:
np.argmax(model.predict(X_test), axis=-1)

array([0, 0, 0, ..., 0, 0, 0])

In [44]:
z=(model.predict(X_test) > 0.5).astype("int32")

In [45]:
z

array([[0],
       [1],
       [0],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [46]:
y_test

array([0, 0, 0, ..., 1, 1, 1])